In [1]:
import sys
import pandas as pd
sys.path.insert(1,'C:\R-Packages')

import rasterio
import numpy as np
import os

# === Specify file paths ===
dtm_path = r'C:\Users\conor.strong\Downloads\wales_dtm_32bit_cog.tif'
dsm_path = r'C:\Users\conor.strong\Downloads\wales_dsm_32bit_cog.tif'
chm_output_path = r'C:\Users\conor.strong\Downloads\Wales_CHM_2020_22.tif'

In [2]:
# === Open source rasters ===
with rasterio.open(dtm_path) as dtm_src, rasterio.open(dsm_path) as dsm_src:
    if (
        dtm_src.width != dsm_src.width or
        dtm_src.height != dsm_src.height or
        dtm_src.transform != dsm_src.transform or
        dtm_src.crs != dsm_src.crs
    ):
        raise ValueError("DTM and DSM rasters are not aligned.")

    meta = dtm_src.meta.copy()
    meta.update(dtype=rasterio.float32)

    with rasterio.open(chm_output_path, 'w', **meta) as chm_dst:
        # Iterate over windows (blocks)
        for ji, window in dtm_src.block_windows(1):
            dtm_block = dtm_src.read(1, window=window)
            dsm_block = dsm_src.read(1, window=window)

            chm_block = dsm_block - dtm_block
            chm_block[chm_block < 0] = 0

            chm_dst.write(chm_block.astype(np.float32), 1, window=window)

print(f"CHM saved successfully at {chm_output_path}")


CHM saved successfully at C:\Users\conor.strong\Downloads\Wales_CHM_2020_22.tif
